In [1]:
import requests
import pandas as pd
from urllib.parse import urlencode


In [2]:
class Services_Awattar_Smartmeter:
    def __init__(self):
        # Awattar Properties
        # self.awattar_json_url = 'https://api.awattar.de/v1/marketdata'
        self.awattar_json_url = "https://api.awattar.at/v1/marketdata?start={}&end={}"
        # self.awattar_json_url = "https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Week?meterId={}&startDate={}&endDate={}"

        
        # Smartmeter Properties
        self.smartmeter_base_url = 'https://smartmeter.netz-noe.at/orchestration/Authentication/Login'
        self.smartmeter_url_credintials = {"user": "SommererPrivatstiftung", "pwd": "SpS*1996"}
        # self.smartmeter_json_url = "https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Day?meterId=AT0020000000000000000000020826368&day={}&__Host-go4DavidSecurityToken={}"
        self.smartmeter_json_url = "https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Week?meterId={}&startDate={}&endDate={}"

    
    
    def get_smartmeter_data(self, meter_id, start_date, end_date):
        auth_cookie = self.post_request(self)
        headers = {
            'Cookie': f'__Host-go4DavidSecurityToken={auth_cookie};',
        }

        meter_id = 'AT0020000000000000000000020826368'
        # Using parameters in the URL
        url = 'https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Week?meterId=AT0020000000000000000000020826368&startDate=2023-9-25&endDate=2023-10-2'
        # url = f'https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Week?meterId={meter_id}&startDate={start_date}&endDate={end_date}'

        # Making a request
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            try:
                smartmeter_json_response = response.json()
            except ValueError:  # includes simplejson.decoder.JSONDecodeError
                print('Decoding JSON has failed')
                print('Response text:', response.text)
                return None

            smartmeter_json_response = pd.DataFrame(smartmeter_json_response)[['meteredValues', 'peakDemandTimes']]
            return smartmeter_json_response
        
        else:
            print(f"Request failed with status code {response.status_code}: {response.text}")
            return None
    
    @staticmethod
    def post_request(self):
        auth_cookie, auth_xsrf_token, nsc_wt = None, None, None
        auth_response = requests.post(self.smartmeter_base_url, json=self.smartmeter_url_credintials)
        if auth_response.status_code == 200:
            print("Authentication successful...")
            auth_cookie = auth_response.cookies['__Host-go4DavidSecurityToken']
            auth_xsrf_token = auth_response.cookies['XSRF-Token']
            nsc_wt = auth_response.cookies['NSC_WT_TWYUXFCQ-TTM']
            return auth_cookie

    def get_awattar_data(self, start_date, end_date): 
        """
        INPUT:
            start_date: str --> 'YYYY:MM:dd hh:mm:ss'
            end_date: str --> 'YYYY:MM:dd hh:mm:ss'
        PARAMS: 
            self.awattar_json_url
        OUTPUT: awattar_json_response: pd.DataFrame
        """
        # Get Awattar Data
        json_url = self.awattar_json_url.format(start_date, end_date)
        awattar_json_response = requests.get(json_url).json()
        awattar_json_response = pd.json_normalize(awattar_json_response['data'])
        awattar_json_response['start_timestamp'] = pd.to_datetime(awattar_json_response['start_timestamp'], unit='ms')
        awattar_json_response['end_timestamp'] = pd.to_datetime(awattar_json_response['end_timestamp'], unit='ms')
        return awattar_json_response
    
     

In [4]:
cls = Services_Awattar_Smartmeter()

cls.get_awattar_data('1693526400000', '1696118399000')

,start_timestamp,end_timestamp,marketprice,unit
0,2023-09-01 00:00:00,2023-09-01 01:00:00,93.29,Eur/MWh
1,2023-09-01 01:00:00,2023-09-01 02:00:00,92.09,Eur/MWh
2,2023-09-01 02:00:00,2023-09-01 03:00:00,91.80,Eur/MWh
3,2023-09-01 03:00:00,2023-09-01 04:00:00,95.98,Eur/MWh
4,2023-09-01 04:00:00,2023-09-01 05:00:00,111.93,Eur/MWh
...,...,...,...,...
714,2023-09-30 18:00:00,2023-09-30 19:00:00,133.18,Eur/MWh
715,2023-09-30 19:00:00,2023-09-30 20:00:00,111.90,Eur/MWh
716,2023-09-30 20:00:00,2023-09-30 21:00:00,107.40,Eur/MWh
717,2023-09-30 21:00:00,2023-09-30 22:00:00,101.04,Eur/MWh


In [ ]:
sep_data = cls.sm('1693526400000', '1696118399000')

In [96]:
sep_data.to_csv('sep_data.csv', index=False)

In [97]:
from urllib.parse import urlencode, quote_plus

# Your parameters
params = {
    'meterId': 'AT0020000000000000000000020826368',
    'startDate': '2021-09-01 09:00:00',
    'endDate': '2021-09-30 09:00:00'
}

# Base URL
base_url = "https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Week"

# Constructing the full URL
url = f"{base_url}?{urlencode(params)}"

print(url)


https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Week?meterId=AT0020000000000000000000020826368&startDate=2021-09-01+09%3A00%3A00&endDate=2021-09-30+09%3A00%3A00


In [98]:
cls.get_smartmeter_data('AT0020000000000000000000020826368', '2023-09-01', '2023-09-29')

Authentication successful...
Request failed with status code 401: 
